# RCy3 Use Cases
## Alex Pico, Julia Gustavsen, Shraddha Pai, Ruth Isserlin, Barry Demchak
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/R/colab/RCy3-use-cases.ipynb)

# Operation
In order to work with RCy3 you must have Cytoscape v3.7 or later installed and running.  Cytoscape can be installed from [cytoscape.org](https://cytoscape.org). The RCy3 package can be installed from [Bioconductor](https://bioconductor.org/packages/release/bioc/html/RCy3.html}{Bioconductor):

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("RCy3")
library(RCy3)

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)

In [ ]:
# Connect to local Cytoscape
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)

Launch Cytoscape and keep it running whenever using RCy3. Confirm that you have everything installed and that RCy3 is communicating with Cytoscape via CyREST:

In [ ]:
cytoscapePing ()
#[1] "You are connected to Cytoscape!"

As with any R package, one can access the documentation and browse over a dozen vignettes included in the RCy3 package:

In [ ]:
help(package=RCy3)
browseVignettes("RCy3")

# Use Cases
The following sections demonstrate a variety of common and advanced network biology use cases as runnable R code snippets. The first set focuses on fundamental Cytoscape operations that are common to most use cases:

* Loading networks (from R objects, Cytoscape files and public databases)
* Visualizing network data
* Filtering by node degree or data
* Saving and exporting networks

Additionally, there are examples that demonstrate analytical workflows, relying not only on Cytoscape, but also on Cytoscape apps and other R packages:

* Building maps of enrichment analysis results using EnrichmentMap and AutoAnnotate
* Visualizing integrated network analysis using BioNet
* Performing advanced graph analytics using RBGL

## Loading Networks
Networks come in all shapes and sizes in multiple formats from multiple sources. Here are just a few of the myriad ways to load networks into Cytoscape using RCy3.

From R Objects. . .

In [ ]:
# From graph objects (graphNEL)  
g <- makeSimpleGraph()
createNetworkFromGraph(g)
## And round-trip back from Cytoscape to graph 
g2 <- createGraphFromNetwork()

# From igraph objects
library(igraph)
ig <- make_graph("Zachary")
createNetworkFromIgraph(ig)
## And round-trip back from Cytoscape to igraph
ig2 <- createIgraphFromNetwork()
## Note that the Cytoscape model infers directionality

# From dataframes
nodes <- data.frame(id=c("node 0","node 1","node 2","node 3"),
                    group=c("A","A","B","B"), #categorical strings
                    score=as.integer(c(20,10,15,5)), #integers
                    stringsAsFactors=FALSE)
edges <- data.frame(source=c("node 0","node 0","node 0","node 2"),
                    target=c("node 1","node 2","node 3","node 3"),
                    interaction=c("inhibits","interacts", 
                                  "activates","interacts"),  #optional
                    weight=c(5.1,3.0,5.2,9.9), #numerics
                    stringsAsFactors=FALSE)
createNetworkFromDataFrames(nodes, edges)

From Cytoscape-supported File Formats. . .

In [ ]:
# From Cytoscape session files
## Will erase and replace all data from current session!
openSession() # default file = galFiltered.cys

# From local network files
importNetworkFromFile() # default file = galFiltered.sif
## Supported file formats: SIF, GML, xGMML, graphML, CX, plus

# From NDEx, the network database
importNetworkFromNDEx("5be85817-1e5f-11e8-b939-0ac135e8bacf")
## Account information or accessKey are required arguments only
## when accessing private content

From Public Databases via Cytoscape Apps. . .

In [ ]:
# From STRING, starting with a list of genes/proteins
installApp("stringApp")
gene.list <- c("T53","AKT1","CDKN1A")
gene.str <- paste(gene.list, collapse = ",")
string.cmd <- paste("string protein query cutoff=0.99 limit=40 query",
                    gene.str, sep = "=")
commandsRun(string.cmd)

# From WikiPathways, starting with a keyword
library(rWikiPathways) # install from Bioconductor
installApp("WikiPathways")
keyword <- "glioblastoma"
gbm.pathways <- findPathwaysByText(keyword)
gbm.wpid <- gbm.pathways[[1]]$id # let’s just take the first one
wikipathways.cmd <- paste("wikipathways import-as-pathway id",
                          gbm.wpid, sep = "=")
commandsRun(wikipathways.cmd)

## Visualizing Data on Networks
Cytoscape excels at generating publication-quality network visualization with data overlays. This vignette demonstrates just one of the hundreds of visual style mapping options using RCy3.

In [ ]:
# Load sample network
closeSession(FALSE) # clears all session data wihtout saving
importNetworkFromFile() # default file = galFiltered.sif

# Load sample data
csv <- system.file("extdata","galExpData.csv", package="RCy3")
data <- read.csv(csv, stringsAsFactors = FALSE)
loadTableData(data,data.key.column="name")

# Prepare data-mapping points
gal80Rexp.min <- min(data$gal80Rexp, na.rm = TRUE)
gal80Rexp.max <- max(data$gal80Rexp, na.rm = TRUE)
## For a balanced color gradient, pick the largest absolute value
gal80Rexp.max.abs <- max(abs(gal80Rexp.min), abs(gal80Rexp.max)) 

# Set node color gradient from blue to white to red
setNodeColorMapping('gal80Rexp', c(-gal80Rexp.max.abs, 0, gal80Rexp.max.abs), 
                    c('#5577FF','#FFFFFF','#FF7755'), default.color = '#999999')


## Filtering Networks by Degree and by Data
Network topology and associated node or edge data can be used to make selections in Cytoscape that enable filtering and subnetworking. The filters are added to the Select tab in the Control Panel of Cytoscape’s GUI and saved in session files.

In [ ]:
# Load demo Cytoscape session file
openSession() # default file = galFiltered.cys
net.suid <- getNetworkSuid() # get SUID for future reference

# Filter for neighbors of high degree nodes
createDegreeFilter(filter.name = "degree filter",
                   criterion = c(0,9),
                   predicate = "IS_NOT_BETWEEN")
selectFirstNeighbors() # expand selection to first neighbors
createSubnetwork(subnetwork.name = "first neighbors of high degree nodes")

# Filter for high edge betweenness
createColumnFilter(filter.name = "edge betweenness",
                   type = "edges",
                   column = "EdgeBetweenness",
                   4000,
                   "GREATER_THAN",
                   network = net.suid)
createSubnetwork(subnetwork.name = "high edge betweenness")

## Saving and Exporting Networks
There are local and cloud-hosted options for saving and sharing network models and images. The Cytoscape session file (CYS) will include all networks, collections, tables and styles. It should retain every aspect of your session, including the size of the application window. Network and image exports include only the currently active network. Export to NDEx requires account information you can obtain from [ndexbio.org](https://ndexbio.org).

In [ ]:
# Saving sessions
saveSession("MySession") #.cys
## Leave filename blank to update previously saved session file

# Exporting images and networks
exportNetwork() #.sif
## Optionally specify filename, default is network name
## Optionally specify type: SIF(default), CX, cyjs, graphML, NNF, SIF, xGMML
exportImage(type='png') #.png
## Optionally specify filename, default is network name
## Optionally specify type: PNG (default), JPEG, PDF, PostScript, SVG 

# Exporting to NDEx, a.k.a. “Dropbox” for networks
exportNetworkToNDEx(username, password, TRUE)
## Account information (username and password) is required to upload
## Use updateNetworkInNDEx if the network has previously been uploaded

## Building Maps of Enrichment Analysis Results
This workflow illustrates how to plot an annotated map of enrichment results using the  [EnrichmentMap Pipeline Collection of apps](http://apps.cytoscape.org/apps/enrichmentmappipelinecollection) in Cytoscape.  An enrichment map is a network visualization of related genesets in which nodes are gene sets (or pathways) and edge weight indicates the overlap in member genes. Following the construction of the enrichment map, AutoAnnotate clusters redundant gene sets and uses WordCloud to label the resulting cluster. The code uses the Commands interface to invoke EnrichmentMap and AutoAnnotate apps. After installing apps, run commandsAPI() to open the live Swagger documentation to browse and execute command-line syntax.

In [ ]:
installApp("EnrichmentMap Pipeline Collection") # installs 4 apps
# Download sample gmt file of human pathways
gmt.file <- "rcy3_enrichmentmap.gmt"
download.file(file.path("http://download.baderlab.org/EM_Genesets",
                        "September_01_2019/Human/symbol/Pathways",
                        "Human_WikiPathways_September_01_2019_symbol.gmt"),
              gmt.file)
# Run EnrichmentMap build command
em_command <- paste('enrichmentmap build analysisType="generic"',
                    "gmtFile=", paste(getwd(), gmt.file, sep="/"),
                    "pvalue=", 1,
                    "qvalue=", 1,
                    "similaritycutoff=",0.25,
                    "coefficients=","JACCARD")
print(em_command)
commandsGET(em_command)
# Run the AutoAnnotate command
aa_command <- paste("autoannotate annotate-clusterBoosted",
                    "clusterAlgorithm=MCL",
                    "labelColumn=EnrichmentMap::GS_DESCR",
                    "maxWords=3")
print(aa_command)
commandsGET(aa_command)
# Annotate a subnetwork
createSubnetwork(c(1:4),"__mclCluster")
commandsGET(aa_command)

## Visualizing Integrated Network Analysis Using BioNet 
The [BioNet](https://bioconductor.org/packages/release/bioc/html/BioNet.html) package implements analytical methods to perform integrated network analysis, e.g., of gene expression data and clinical survival data in the context of protein-protein interaction networks. Partnered with RCy3, the analytical results from BioNet can be visualized in Cytoscape with vastly more options for customization. Starting with the ["Quick Start" tutorial](https://bioconductor.org/packages/release/bioc/vignettes/BioNet/inst/doc/Tutorial.pdf) from BioNet, we pass the results directly to Cytoscape for visualization:

In [ ]:
library(BioNet) # install from Bioconductor
library(DLBCL) # install from Bioconductor
data(dataLym)
data(interactome)
## The following steps are from BioNet's Quick Start tutorial:
pvals <- cbind(t = dataLym$t.pval, s = dataLym$s.pval)
rownames(pvals) <- dataLym$label
pval <- aggrPvals(pvals, order = 2, plot = FALSE)
subnet <- subNetwork(dataLym$label, interactome)
subnet <- rmSelfLoops(subnet)
fb <- fitBumModel(pval, plot = FALSE)
scores <- scoreNodes(subnet, fb, fdr = 0.001)
module <- runFastHeinz(subnet, scores)
logFC <- dataLym$diff
names(logFC) <- dataLym$label
plotModule(module, scores = scores, diff.expr = logFC)

# Using RCy3 we can generate a custom visualization of the same output
## Create network from graphNEL object and load data calculated above
createNetworkFromGraph(module, "module", "BioNet")
loadTableData(as.data.frame(scores))
loadTableData(as.data.frame(logFC))
## Set styles
setNodeLabelMapping("geneSymbol")
setNodeFontSizeDefault(18)
setNodeBorderWidthDefault(3.0)
logFC.max.abs <- max(abs(min(logFC)), abs(max(logFC))) 
setNodeColorMapping('logFC', c(-logFC.max.abs, 0, logFC.max.abs), 
                    c('#5577FF','#FFFFFF','#FF7755'), default.color = '#999999')
createColumnFilter("Positive scores", "scores",c(0,max(scores)),"BETWEEN")
setNodeShapeBypass(getSelectedNodes(), "ELLIPSE")
clearSelection()

##Performing Advanced Graph Analytics Using RBGL
As an interface to the BOOST library, the [RBGL](https://bioconductor.org/packages/release/bioc/html/RBGL.html) Bioconductor package offers an impressive array of analytical functions for graphs. Here we will start with a network in Cytoscape, load it into R as a graph object, perform shortest path calculation using RBGL and then visualize the results back in Cytoscape.

In [ ]:
library(RBGL) # install from Bioconductor
# Convert a sample Cytoscape network to a graph object
openSession()
g <- createGraphFromNetwork()
# Identify start and finish nodes (styling is optional)
start <- "YNL216W"
finish <- "YER040W"
setNodeBorderWidthBypass(c(start, finish), 20)
setNodeBorderColorBypass(start, "#00CC33")
setNodeBorderColorBypass(finish, "#CC00CC")
# Use RBGL to perform shortest path calculation
shortest <- sp.between(g, start, finish)
shortest$`YNL216W:YER040W`$length
#[1] 6
shortest.path <- shortest$`YNL216W:YER040W`$path_detail
# Visualize results in Cytoscape
selectNodes(shortest.path, "name")
setNodeBorderWidthBypass(shortest.path, 20)
createSubnetwork()